In [21]:
%matplotlib inline
from sklearn import neighbors
from nltk.stem.wordnet import WordNetLemmatizer
import string
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn import decomposition
import scipy
import sklearn
import spacy
from sklearn.model_selection import train_test_split
import re
import os
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from nltk.corpus import gutenberg, stopwords
from collections import Counter

# NLP for scientific articles
This project is aimes at clustering scientific articles.

## Why are we interested?
There is so much scientific information online and countless resources to gain insights into scientific problems that I wanted to create a way to organize information based topic. This will help take disjointed information on the same topic and build resource databases

## Why this dataset?
Science is one of the fields where I believe data science can dramatically increase productivity and discovery by consolidating information and making it easier to access. I also wanted to create a model that succeeds in clustering articles within the same field.

## What are we trying to achieve?

The goal is to create a filter for articles and/or scientific documentation that can be used to unify scientific information. Once this model is successful it would make sense to create sub categories within the parent categories and continue to do so until we can easily access specific topics.

In [22]:
article_list = []
true_labels = []
categories = ['business', 'entertainment', 'politics', 'sport',  'tech']
for i, category in enumerate(categories):
    for root, dirs, files in os.walk(os.path.join("./bbc",category)):
        for file in sorted(files):
            true_labels.append(i)
            try:
                with open(os.path.join(root,file),"r",encoding='utf-8') as f:
                    article_list.append(f.read())
            except:
                with open(os.path.join(root,file),"r",encoding='iso-8859-1') as f:
                    article_list.append(f.read())
print(article_list[0])

IndexError: list index out of range

In [23]:
# Tokenizing 
nlp = spacy.load('en')
stream = nlp.pipe(article_list)

In [17]:
# Converting tokens to sentences and lemmas. Assuring punctuation is scrubbed
sentences = []
docs = []
for doc in stream:
    docs.append(doc)
    for sentence in doc.sents:
        sentence = [
            token.lemma_
            for token in sentence
            if not token.is_stop
            and not token.is_punct
        ]
        sentences.append(sentence)


#print(sentences)
print('We have {} sentences'.format(len(sentences)))

We have 0 sentences


In [18]:
#------------------------------------------CBOW word2vec-----------------------------------------------
# Attempting to group words together based on proximity. Using CBOW due to smaller corpus
cbow_model = word2vec.Word2Vec(
    sentences,     
    min_count=5,   # Minimum word count threshold.
    window=5,     # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=100,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)


RuntimeError: you must first build vocabulary before training the model

In [ ]:
# word2vec analysis 
vocab = cbow_model.wv.vocab.keys()

print(cbow_model.wv.most_similar(positive=['woman', 'man'], negative=['boy']))
# Testing to see which words it will group with 'planet'
#print('Similarity to planet')
print('1:')
print(cbow_model.wv.most_similar('politician'))
print('2:')
print(cbow_model.wv.most_similar('scientist'))
print('3:')
print(cbow_model.wv.most_similar('artist'))
print('4:')
print(cbow_model.wv.most_similar('musician'))

# Testing for similarity 
#print('Cosine similatrity analysis:')
#print('study / research')
print('5:')
print(cbow_model.wv.similarity('study', 'research'))
#print('court / field')
print('6:')
print(cbow_model.wv.similarity('computer', 'tool'))
print('7:')
print(cbow_model.wv.similarity('musician', 'artist'))
print('8:')
print(cbow_model.wv.similarity('phone', 'device'))

# Testing to see if it will leave out the right word (looking for 'dinosaur')
#print('Which doesnt fit?')
#print('bat / bird / dinosaur / bee')
print(cbow_model.doesnt_match("man woman plane dog".split()))
print(cbow_model.doesnt_match("rock human computer dog".split()))
print(cbow_model.doesnt_match("rock human rocket dog".split()))
print(cbow_model.doesnt_match("car human rocket dog".split()))
print(cbow_model.doesnt_match("car human rocket boat".split()))

#function that adds up vectors, normalize vector?'''
# compare
# study / molecule
# atom / research
# unrelated
# more doesn't match
# dissimilar

In [ ]:

x = cbow_model.wv['human']
print(len(x))
vector_list = []
for doc in docs:
    article_vector = np.zeros_like(x)
    for w in doc:
        word = w.lemma_
        if word in cbow_model.wv:
            vector = cbow_model.wv[word]
            article_vector += vector
    norm = normalize(article_vector.reshape(1, -1))
    norm = norm.flatten()
    vector_list.append(norm)


In [20]:
clusters = [2, 3, 5, 8, 10, 15, 20]
for num cluster in clusters:
    X_w2v = vector_list
    y_pred_w2v = KMeans(n_clusters=clusters).fit_predict(X_w2v)
    model = KMeans(n_clusters=clusters).fit(X_w2v)
    df = pd.DataFrame()
    df['pred'] = y_pred_w2v
    df['labels'] = true_labels
df.tail()

NameError: name 'vector_list' is not defined

In [ ]:
print('Silhouette Score')
print(metrics.silhouette_score(X_w2v, df['pred'], metric='euclidean'))
#array = X.toarray()
print('Calinski Harabaz Index')
print(metrics.calinski_harabaz_score(X_w2v, df['pred']))
print('Opposite of the value of X on the K-means objective.')
print(model.score(X_w2v))
print('Adjusted Rand Index')
print(metrics.adjusted_rand_score(df['labels'], df['pred']))

In [19]:
#y_w2v = df['pred']
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, true_labels , test_size=0.1)

NameError: name 'X_w2v' is not defined

In [24]:
knn_w2v = neighbors.KNeighborsClassifier(n_neighbors=n)
knn_w2v.fit(X_train_w2v, y_train_w2v)
y_pred = knn_w2v.predict(X_test_w2v)
print('Percentage accuracy')
print(knn_w2v.score(X_test_w2v, y_test_w2v))

NameError: name 'X_train_w2v' is not defined

In [ ]:
pca = decomposition.PCA(n_components=2)
pca.fit(np.array(X_w2v))
X_2 = pca.transform(X_w2v)

plt.figure(figsize=(12,8))
plt.scatter(X_2[:,0],X_2[:,1],c=df['pred'])
plt.title("Colors indicate clusters")

plt.figure(figsize=(12,8))
plt.scatter(X_2[:,0],X_2[:,1],c=true_labels)
plt.title("Colors indicate True Labels")


In [ ]:
np.array(X_w2v).shape

In [ ]:
#-------------------------------------------tfidf-------------------------------------------------


vectorizer = TfidfVectorizer(max_df=0.25,          # drop words that occur in more than a quarter of the articles
                             min_df=2,             # only use words that appear at least twice
                             stop_words='english', # picking list of stop words
                             lowercase=True,       # convert everything to lower case 
                             use_idf=True,         # use inverse document frequencies in our weighting
                             norm=u'l2',           # Treat longer and shorter paragraphs equally
                             smooth_idf=True       # Prevent divide-by-zero errors
                            )


# Applying the vectorizer
article_list_tfidf = vectorizer.fit_transform(article_list)
print("Number of features: %d" % article_list_tfidf.get_shape()[1])

#X_train_tfidf, X_test_tfidf= train_test_split(article_list_tfidf, test_size=0.4, random_state=0)


# Reshape the vectorizer output to be legible 
article_list_tfidf_csr = article_list_tfidf.tocsr()

# number of articles
n = article_list_tfidf_csr.shape[0]

# Making a list of dictionaries, one per article
tfidf_bypara = [{} for _ in range(0,n)]

# List of features
terms = vectorizer.get_feature_names()

# for each article, lists the feature words and their tf-idf scores
for i, j in zip(*article_list_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = article_list_tfidf_csr[i, j]


print('Original article:', article_list[5])
print('Tf_idf vector:', tfidf_bypara[5])

In [4]:
dict(article_list_tfidf.tocsr()[1])

NameError: name 'article_list_tfidf' is not defined

In [5]:
# Applying SVD to reduce term document matrix into lower dimensional space (from 3417 to 260)
svd= TruncatedSVD(260)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data and projecting the training data.
article__list_lsa = lsa.fit_transform(article_list_tfidf)

# Checking variance explained
variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100) # Upon review seems like overfitting

# Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
# It seems like some similarities exist but needs refining
paras_by_component=pd.DataFrame(article__list_lsa,index=article_list)

for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

NameError: name 'article_list_tfidf' is not defined

In [6]:
# Viewing the vectorizer output
print(article_list_tfidf_csr)

NameError: name 'article_list_tfidf_csr' is not defined

In [7]:
X_tfidf = article_list_tfidf
X_norm_tfidf = normalize(X_tfidf)
model_tfidf = KMeans(n_clusters=5).fit(X_norm_tfidf)
#model_tfidf_labels = model_tfidf.labels_
prediction_tfidf = model_tfidf.predict(X_norm_tfidf)

NameError: name 'article_list_tfidf' is not defined

In [8]:
X_norm_tfidf

NameError: name 'X_norm_tfidf' is not defined

In [9]:
df = pd.DataFrame()
df['pred_tfidf'] = prediction_tfidf
#labels_tfidf.flatten()
df['labels_tfidf'] = true_labels
df.head(50)
df

NameError: name 'prediction_tfidf' is not defined

In [ ]:
print(article_clusters)

In [89]:
print('Silhouette Score')
print(metrics.silhouette_score(X_norm_tfidf, labels, metric='euclidean'))
array = X_norm_tfidf.toarray()
print('Calinski Harabaz Index')
print(metrics.calinski_harabaz_score(array, labels))
print('Opposite of the value of X on the K-means objective.')
print(model_tfidf.score(X_norm_tfidf))
print('Adjusted Rand Index')
print(metrics.adjusted_rand_score(labels, prediction_tfidf))

Silhouette Score
0.0114262569181
Calinski Harabaz Index
14.6964294444
Opposite of the value of X on the K-means objective.
-2126.4025846422614
Adjusted Rand Index
0.644223589628


In [60]:
#y_tfidf = df['pred_tfidf']
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_norm_tfidf, labels, test_size=0.3)

In [62]:
knn_tfidf = neighbors.KNeighborsClassifier()
knn_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred = knn_tfidf.predict(X_test_tfidf)
print('Percentage accuracy')
print(knn_w2v.score(X_test_tfidf, y_test_tfidf))

Percentage accuracy


ValueError: kd_tree does not work with sparse matrices. Densify the data, or set algorithm='brute'

In [91]:
def generate_labels(df):
    predicted_clusters = {}
    for cluster_number in df.iloc[:, 0].unique():
        df_sub = df[df.iloc[:, 0] == cluster_number]
        counts = df_sub.groupby(df.columns[1]).count()
        #print(counts)
        predicted_label = counts.iloc[:, 0].argmax()
        predicted_clusters[cluster_number] = predicted_label
    return predicted_clusters
label_map = generate_labels(df)
df['predicted_label'] = df.iloc[:, 0].map(lambda x: label_map[x])
df.head(50)

        
#Add fourth column with text categories
    

/Users/vlonij/virtual_envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  import sys


,pred_tfidf,labels_tfidf,predicted_label
0,4,1,1
1,4,1,1
2,4,1,1
3,4,1,1
4,4,1,1
5,4,1,1
6,4,1,1
7,4,1,1
8,4,1,1
9,4,1,1


In [92]:
knn_w2v = neighbors.KNeighborsClassifier()
knn_w2v.fit(X_train_w2v, y_train_w2v)
y_pred = knn_w2v.predict(X_test_w2v)
print('Percentage accuracy')
print(knn_w2v.score(X_test_w2v, y_test_w2v))

Percentage accuracy
0.515695067265


In [1]:
pred_label = df['predicted_label']
def generate_label_names(df):
    labels = []
    for _ in pred_label:
        if df['predicted_label'] == 0:
            labels.append('business')
        if df['predicted_label'] == 1:
            labels.append('entertainment')
        if df['predicted_label'] == 2:
            labels.append('politics')
        if df['predicted_label'] == 3:
            labels.append('sports')
    if df['predicted_label'] == 4:
        labels.append('tech')
    return labels
label_names = generate_label_names(df)

NameError: name 'df' is not defined

In [175]:
label_names = generate_label_names(df)
df['doc_label'] = label_names
#df.head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [127]:
business = prediction_tfidf[:510]
print(business)
def business_label(business):
    business_label = ''
    mx = 0
    zero = 0
    one = 0
    two = 0
    three = 0
    four = 0
    for prediction in business:
        if prediction == 0:
            zero += 1
        elif prediction == 1:
            one += 1
        elif prediction == 2:
            two += 1
        elif prediction == 3:
            three += 1
        elif prediction == 4:
            four += 1
        if zero > mx:
            business_label = 'zero'
            mx = zero
        elif one > mx:
            business_label = 'one'
            mx = one
        elif two > mx:
            business_label = 'two'
            mx = two
        elif three > mx:
            business_label = 'three'
            mx = three
        elif four > mx:
            business_label = 'four'
            mx = four
        return business_label
business_label(business)

[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4
 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 1 4 2 4 4 4 4 4 4 4 4 4 4 4
 4 1 4 4 4 4 1 4 4 4 4 2 4 4 4 4 4 4 4 1 2 4 4 4 4 4 4 4 4 2 4 2 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

'four'

In [128]:
entertainment = prediction_tfidf[510:896]
print(entertainment)
def entertainment_label(entertainment):
    entertainment_label = ''
    mx = 0
    zero = 0
    one = 0
    two = 0
    three = 0
    four = 0
    for prediction in entertainment:
        if prediction == 0:
            zero += 1
        elif prediction == 1:
            one += 1
        elif prediction == 2:
            two += 1
        elif prediction == 3:
            three += 1
        elif prediction == 4:
            four += 1
        if zero > mx:
            entertainment_label = 'zero'
            mx = zero
        elif one > mx:
            entertainment_label = 'one'
            mx = one
        elif two > mx:
            entertainment_label = 'two'
            mx = two
        elif three > mx:
            entertainment_label = 'three'
            mx = three
        elif four > mx:
            entertainment_label = 'four'
            mx = four
        return entertainment_label
entertainment_label(entertainment)

[4 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 4 3 4 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 4 3 4 3
 3 4 4 4 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 4 2
 4 3 3 3 3 4 3 3 3 3 3 3 3 3 3 4 3 3 3 4 3 3 4 3 4 3 3 3 3 4 4 3 3 3 3 3 3
 3 3 3 3 3 3 3 4 3 3 3 3 3 4 3 3 2 3 4 4 3 3 3 4 4 4 3 4 3 3 4 3 4 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 3
 3 3 3 3 3 3 2 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 2 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


'four'

In [129]:
politics = prediction_tfidf[896:1313]
print(politics)
def politics_label(politics):
    politics_label = ''
    mx = 0
    zero = 0
    one = 0
    two = 0
    three = 0
    four = 0
    for prediction in politics:
        if prediction == 0:
            zero += 1
        elif prediction == 1:
            one += 1
        elif prediction == 2:
            two += 1
        elif prediction == 3:
            three += 1
        elif prediction == 4:
            four += 1
        if zero > mx:
            politics_label = 'zero'
            mx = zero
        elif one > mx:
            politics_label = 'one'
            mx = one
        elif two > mx:
            politics_label = 'two'
            mx = two
        elif three > mx:
            politics_label = 'three'
            mx = three
        elif four > mx:
            politics_label = 'four'
            mx = four
        return politics_label
politics_label(politics)

[4 4 4 0 0 4 0 0 0 4 0 4 0 0 4 4 4 4 4 4 0 0 4 0 4 4 0 0 4 4 4 4 4 0 4 0 0
 4 4 4 0 4 0 4 4 0 4 4 0 4 4 4 4 0 4 4 4 4 0 4 4 0 4 4 4 4 4 4 4 0 4 0 4 4
 0 4 0 4 4 4 0 4 0 4 4 4 0 0 4 4 4 4 0 4 0 0 4 4 0 4 4 0 4 4 4 0 0 4 2 0 0
 0 4 4 4 4 0 4 4 0 4 0 2 4 4 4 4 0 4 4 4 4 4 4 0 4 4 4 4 4 0 4 0 4 4 4 0 4
 0 4 4 4 0 4 4 4 0 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 0 4 0 4 0 4
 0 4 4 0 1 4 4 0 4 4 0 4 4 2 0 0 4 4 4 4 0 0 4 4 4 4 0 0 4 4 4 0 0 1 0 4 4
 0 4 0 0 0 0 4 4 4 4 4 4 0 0 4 0 4 4 4 4 4 4 4 4 0 4 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 4 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 4 0 0 0 4 4 4 0 4 0 0
 0 0 0 0 0 0 0 0 0 0 0 4 4 2 4 4 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 4 4 0 4 4 4
 0 0 4 0 0 0 4 0 0 4 4 0 4 4 4 4 0 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 4 0 0 0 0 0]


'four'

In [125]:
sports = prediction_tfidf[1313:1824]
print(sports)
def sports_label(sports):
    sports_label = ''
    mx = 0
    zero = 0
    one = 0
    two = 0
    three = 0
    four = 0
    for prediction in sports:
        if prediction == 0:
            zero += 1
        elif prediction == 1:
            one += 1
        elif prediction == 2:
            two += 1
        elif prediction == 3:
            three += 1
        elif prediction == 4:
            four += 1
        if zero > mx:
            sports_label = 'zero'
            mx = zero
        elif one > mx:
            sports_label = 'one'
            mx = one
        elif two > mx:
            sports_label = 'two'
            mx = two
        elif three > mx:
            sports_label = 'three'
            mx = three
        elif four > mx:
            sports_label = 'four'
            mx = four
        return sports_label
sports_label(sports)

[1 1 1 4 1 1 1 1 1 1 1 1 1 1 4 3 3 1 1 1 1 1 3 1 1 4 3 1 1 1 1 1 3 1 4 4 1
 1 4 1 3 1 1 4 4 1 3 3 4 3 3 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 3 3 3 3 1 1 1 1 1 4 3 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

'one'

In [126]:
tech = prediction_tfidf[1824:]
print(tech)
def tech_label(tech):
    tech_label = ''
    mx = 0
    zero = 0
    one = 0
    two = 0
    three = 0
    four = 0
    for prediction in tech:
        if prediction == 0:
            zero += 1
        elif prediction == 1:
            one += 1
        elif prediction == 2:
            two += 1
        elif prediction == 3:
            three += 1
        elif prediction == 4:
            four += 1
        if zero > mx:
            tech_label = 'zero'
            mx = zero
        elif one > mx:
            tech_label = 'one'
            mx = one
        elif two > mx:
            tech_label = 'two'
            mx = two
        elif three > mx:
            tech_label = 'three'
            mx = three
        elif four > mx:
            tech_label = 'four'
            mx = four
        return tech_label
tech_label(tech)

[2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 4 2 2 2 2 4 2 4 2 2
 2 2 2 2 2 2 3 2 4 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 4 2 2 2 2 2 2 2 3 2 3 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2
 4 2 2 2 2 4 2 2 2 2 2 1 3 2 2 2 2 2 4 4 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 4 2 2 2 4 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 4 2 2 2 2 2 2 4 2 2 2 2 4 2 2 2
 2 2 2 2 1 2 2 2 2 2 2 4 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 4 2 2
 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


'two'

In [169]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_norm_tfidf, y, test_size=0.3)

In [171]:
knn_tfidf = neighbors.KNeighborsClassifier()
knn_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred = knn_tfidf.predict(X_test_tfidf)
print('Percentage accuracy')
print(knn_tfidf.score(X_test_tfidf, y_test_tfidf))

Percentage accuracy
0.913043478261


In [173]:
labels = y
print('Silhouette Score')
print(metrics.silhouette_score(X_norm_tfidf, labels, metric='euclidean'))
array = X_norm_tfidf.toarray()
print('Calinski Harabaz Index')
print(metrics.calinski_harabaz_score(array, labels))
print('Opposite of the value of X on the K-means objective.')
print(model_tfidf.score(X_norm_tfidf))
print('Adjusted Rand Index')
print(metrics.adjusted_rand_score(labels_tfidf, y))

Silhouette Score
0.012023046525
Calinski Harabaz Index
14.6238562307
Opposite of the value of X on the K-means objective.
-2123.6796809241637
Adjusted Rand Index


ValueError: labels_true must be 1D: shape is (2223, 1)